In [8]:
import string
import re
printable = set(string.printable)
import sqlite3
import sys

In [9]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
 
    return None

In [10]:
conn = create_connection("./nasb.db")

create_table_sql = """ CREATE TABLE IF NOT EXISTS nasb (
                                        ref text PRIMARY KEY,
                                        book text NOT NULL,
                                        text text NOT NULL,
                                        chapter integer NOT NULL,
                                        verse integer NOT NULL
                                    ); """
c = conn.cursor()
c.execute(create_table_sql)

In [11]:
f = open("./nasb.txt", "r")
data = f.read()
verses = []
for verse in data.split('\n')[9:]:
    verses.append(''.join(filter(lambda x: x in printable, verse)))

In [12]:
bible = []
conn = create_connection('./nasb.db')
c = conn.cursor()
for verse in verses:
    if re.search("\d{1,2}:\d{1,3}|\d{1,2}:\d{1,2}", verse):
        index = re.search("\d{1,2}:\d{1,3}|\d{1,2}:\d{1,2}", verse).span()[1]
        ref = verse[:index]
        ref = ref.split(' ')
        book_data = ref[:len(ref)-1]
        book = ''
        for word in book_data:
            book += word + ' '
        book = book.strip()
        chapter = int(ref[len(ref)-1].split(':')[0])
        verse_num = int(ref[len(ref)-1].split(':')[1])
        body = verse[index:].strip()
        ref = book + ' ' + str(chapter) + ':'+str(verse_num)
        v = {'Book':book, 'Chapter':chapter, 'Verse': verse_num, 'Text': body, 'Ref': ref}
        bible.append(v)
        body = body.replace("'", "''")
        body = body.replace('"', '""')
        sql = ''' INSERT INTO nasb(ref,book,text,chapter,verse)
              VALUES("{0}","{1}","{2}",{3},{4}) '''.format(ref, book, body.replace('"', '""'), chapter, verse_num)
        c.execute(sql)
        conn.commit()
        sys.stdout.write(ref + '    \r')
        sys.stdout.flush()
        
conn.close()

In [20]:
conn = create_connection('./nasb.db')
sql = '''
SELECT Ref, text book from nasb where book = "Matt"'''
c = conn.cursor()
c.execute(sql)
rows = c.fetchall()
for row in rows:
    print (row)
print(len(rows))

('Matt 1:1', 'The record of the genealogy of Jesus the Messiah, the son of David, the son of Abraham:')
('Matt 1:2', 'Abraham was the father of Isaac, Isaac the father of Jacob, and Jacob the father of Judah and his brothers.')
('Matt 1:3', 'Judah was the father of Perez and Zerah by Tamar, Perez was the father of Hezron, and Hezron the father of Ram.')
('Matt 1:4', 'Ram was the father of Amminadab, Amminadab the father of Nahshon, and Nahshon the father of Salmon.')
('Matt 1:5', 'Salmon was the father of Boaz by Rahab, Boaz was the father of Obed by Ruth, and Obed the father of Jesse.')
('Matt 1:6', 'Jesse was the father of David the king.  David was the father of Solomon by Bathsheba who had been the wife of Uriah.')
('Matt 1:7', 'Solomon was the father of Rehoboam, Rehoboam the father of Abijah, and Abijah the father of Asa.')
('Matt 1:8', 'Asa was the father of Jehoshaphat, Jehoshaphat the father of Joram, and Joram the father of Uzziah.')
('Matt 1:9', 'Uzziah was the father of Jot